In [1]:
from shearletNN.shearlets import getcomplexshearlets2D
from shearletNN.shearlet_utils import frequency_shearlet_transform, spatial_shearlet_transform, ShearletTransformLoader, shifted_frequency_shearlet_transform
from shearletNN.complex_resnet import complex_resnet18, complex_resnet34, complex_resnet50
from shearletNN.layers import CGELU, CReLU

import torch
import torchvision
from torchvision.transforms import v2
from torchvision import transforms

import gc


patch_size = 32
image_size = 32

rows, cols = image_size, image_size


shearlets, shearletIdxs, RMS, dualFrameWeights = getcomplexshearlets2D(	rows, 
                                                                        cols, 
                                                                        1, 
                                                                        3, 
                                                                        1, 
                                                                        0.5,
                                                                        wavelet_eff_support = image_size,
                                                                        gaussian_eff_support = image_size
                                                                        )

shearlets = torch.tensor(shearlets).permute(2, 0, 1).type(torch.complex64).to(0)

In [2]:
class Unraveling:
    def __init__(self, n):
        self.levels = []
        for i in range(0, n // 2):
            level = []
            for j in range(i, n - i):
                level.append((j, i))
                level.append((i, j))

                level.append((j, n - (i + 1)))
                level.append((n - (i + 1), j))

            level = list(set(level))
            self.levels.append((torch.tensor([x for x, _ in level]), torch.tensor([y for _, y in level])))
    
    def __call__(self, x):
        return [x[..., a, b] for a, b in self.levels]

# this is still technically resolution-independent because that property comes from the shearlet transform crop
# in this case we do not actually need variable number of input tokens because we are always extracting a finite amount of information...
# if the whole premise is that we can utilize even larger images then why does it matter that we are using a variable size input?
class Freakformer(torch.nn.Module):
    def __init__(self, n, chans, embed_dim = 384):
        # first we need to build the tokenizer using the Unraveling transformation and some linear layers
        # this is equivalent to the patching layer
        self.unravel = Unraveling(n)
        self.layers = torch.nn.ModuleList([torch.nn.Linear(len(level), embed_dim) for level in self.unravel.layers])
        pass

    def forward(self, x):
        unraveled = self.unravel(x)
        # (B, C, 4 ... 8(n // 2) + 4)
        tokens = torch.stack([layer(level.flatten(1)) for layer, level in zip(self.layers, unraveled)], -2)
        # (B, n // 2, embed_dim)
        pass

In [3]:
from tqdm import tqdm

def train(model, optimizer, loader, accumulate=1):
    model.train()
    loss = torch.nn.CrossEntropyLoss()

    for i, (X, y) in tqdm(enumerate(loader)):
        out = model(X.to(0))
        optimizer.zero_grad()
        l = loss(out, y.to(0)) / accumulate
        l.backward()
        if i % accumulate == (accumulate - 1):
            optimizer.step()
        

def accuracy(output, target, topk=(1,)):
    output = output.to(torch.device('cpu'))
    target = target.to(torch.device('cpu'))
    maxk = max(topk)
    batch_size = target.shape[0]

    _, idx = output.sort(dim=1, descending=True)
    pred = idx.narrow(1, 0, maxk).t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(dim=0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def epoch_accuracy(loader_s, student):
    student.eval()

    out_epoch_s = [accuracy(student(L.to(0)), y)[0].detach().cpu().item() for L, y in loader_s]

    student.train()

    return sum(out_epoch_s) / len(out_epoch_s)

def test(network, test_loader):
    network.eval().to(0)
    test_loss = 0
    correct = 0
    total = 0
    test_losses=[]
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data.to(0))
            test_loss += torch.nn.CrossEntropyLoss()(output, target.to(0)).item()
            pred = output.data.max(1, keepdim=True)[1].cpu()
            correct += pred.eq(target.data.view_as(pred)).sum()
            total += target.shape[0]
        test_loss /= total
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, total,
        100. * correct / total))

class IndexSubsetDataset:
    def __init__(self, ds, inds):
        self.ds = ds
        self.inds = inds

    def __iter__(self):
        for i in range(len(self.inds)):
            yield self[i]
    
    def __getitem__(self, i):
        return self.ds[self.inds[i]]
    
    def __len__(self):
        return len(self.inds)
    
    
class FreqModel(torch.nn.Module):
    def __init__(
        self,
        img_size = 224,
        in_chans = 3,
        embed_dim = 384,
    ):
        super().__init__()
        self.img_size = (img_size, img_size)
        self.grid_size = (img_size // 2, 1)
        self.num_patches = img_size // 2

        self.unravel = Unraveling(img_size)
        self.layers = torch.nn.ModuleList([torch.nn.Linear(len(level[0]) * in_chans, embed_dim, dtype=torch.complex64) for level in self.unravel.levels])

    def forward(self, x):

        unraveled = self.unravel(x)
        print([(level.flatten(1).shape, layer.weight.shape) for layer, level in zip(self.layers, unraveled)])
        tokens = torch.cat([layer(level.flatten(1)) for layer, level in zip(self.layers, unraveled)], -2)
        
        return x
    

def linearleaves(module):
    # returns a list of pairs of (parent, submodule_name) pairs for all submodule leaves of the current module
    if isinstance(module, torch.nn.Linear):
        return [(module, None)]

    linear_children = []
    for name, mod in module.named_modules():
        if isinstance(mod, torch.nn.Linear):
            linear_children.append((name, module))
    return linear_children

In [4]:
batch_size_train = 256

In [5]:
def linearleaves(module):
    # returns a list of pairs of (parent, submodule_name) pairs for all submodule leaves of the current module
    if isinstance(module, torch.nn.Linear):
        return [(module, None)]

    linear_children = []
    for name, mod in module.named_modules():
        if isinstance(mod, torch.nn.Linear) or isinstance(mod, torch.nn.Conv2d):
            linear_children.append((name, module))
    return linear_children
        

def getattrrecur(mod, s):
    s = s.split('.')
    for substr in s:
        mod = getattr(mod, substr)
    return mod


def setattrrecur(mod, s, value):
    s = s.split('.')
    for substr in s[:-1]:
        mod = getattr(mod, substr)
    setattr(mod, s[-1], value)


def spectral_normalize(model):
    for name, mod in linearleaves(model):
        setattrrecur(model, name, torch.nn.utils.parametrizations.spectral_norm(getattrrecur(mod, name)))
    
    return model

In [6]:
from shearletNN.complex_deit import Attention, vit_models, Block, FreqEmbed, complex_freakformer_small_patch1_LS, complex_freakformer_small_patch2_LS
from shearletNN.layers import CReLU, ComplexLayerNorm
from functools import partial

def repeat3(x):
    return x.repeat(3, 1, 1)[:3]

train_transform = v2.Compose([
    transforms.RandomResizedCrop((image_size, image_size), scale=(0.5, 1.0)),
    # transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = v2.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

ds_train = torchvision.datasets.CIFAR10('../', transform=train_transform, download = True, train=True)

ds_val = torchvision.datasets.CIFAR10('../', transform=val_transform, download = True, train=False)

train_loader = torch.utils.data.DataLoader(
  ds_train,
  batch_size=batch_size_train, shuffle=True, num_workers=0)

shearlets = shearlets[:3]

def shearlet_transform(img):
    return shifted_frequency_shearlet_transform(img.to(0), shearlets, patch_size)

train_loader = ShearletTransformLoader(train_loader, shearlet_transform)

val_loader = torch.utils.data.DataLoader(
  ds_val,
  batch_size=batch_size_train, shuffle=False)

val_loader = ShearletTransformLoader(val_loader, shearlet_transform)

#for x, y in tqdm(train_loader):
#    assert list(x.shape) == [batch_size_train, shearlets.shape[0] * 3, patch_size, patch_size], x.shape
#    break
print('building model...')

model = spectral_normalize(complex_freakformer_small_patch2_LS(
    img_size=patch_size,
    in_chans=shearlets.shape[0] * 3,
))

# model = complex_resnet18(in_dim=shearlets.shape[0] * 3)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('training model...')
for epoch in range(128):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 8 == 7:
        test(model, train_loader)
        test(model, val_loader)

/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


Files already downloaded and verified
Files already downloaded and verified
building model...
training model...
epoch 0


391it [00:36, 10.58it/s]


epoch 1


391it [00:37, 10.50it/s]


...

epoch 110


391it [00:36, 10.74it/s]


epoch 111


86it [00:07, 10.76it/s]


KeyboardInterrupt: 

In [6]:
from shearletNN.complex_deit import vit_models, LPatchEmbed, Attention, Block, ComplexLayerNorm, complex_Lfreakformer_small_patch1_LS
from shearletNN.layers import CReLU, ComplexLayerNorm
from functools import partial
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

patch_size = 32
image_size = 32

def repeat3(x):
    return x.repeat(3, 1, 1)[:3]

train_transform = v2.Compose([
    transforms.RandomResizedCrop((image_size, image_size), scale=(0.5, 1.0)),
    # transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = v2.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

ds_train = torchvision.datasets.Caltech101('./', transform=train_transform, download = True)
ds_train = IndexSubsetDataset(ds_train, sum([list(range(len(ds_train)))[i::5] for i in range(1, 5)], []))

ds_val = torchvision.datasets.Caltech101('./', transform=val_transform, download = True)
ds_val = IndexSubsetDataset(ds_val, list(range(len(ds_val)))[0::5])

ds_train = torchvision.datasets.CIFAR10('../', transform=train_transform, download = True, train=True)

ds_val = torchvision.datasets.CIFAR10('../', transform=val_transform, download = True, train=False)

train_loader = torch.utils.data.DataLoader(
  ds_train,
  batch_size=batch_size_train, shuffle=True, num_workers=0)

shearlets = shearlets[:3]

def shearlet_transform(img):
    return shifted_frequency_shearlet_transform(img.to(0), shearlets.to(0), image_size)

train_loader = ShearletTransformLoader(train_loader, shearlet_transform)

val_loader = torch.utils.data.DataLoader(
  ds_val,
  batch_size=batch_size_train, shuffle=False)

val_loader = ShearletTransformLoader(val_loader, shearlet_transform)

for x, y in tqdm(train_loader):
    assert list(x.shape) == [batch_size_train, shearlets.shape[0] * 3, image_size, image_size], x.shape
    break
print('building model...')

model = complex_Lfreakformer_small_patch1_LS(img_size = image_size, in_chans = shearlets.shape[0] * 3)


with torch.no_grad():
    for param in model.parameters():
        if not ((param == 0).all() or (param == 1).all()):
            param.data /= 2**(0.5)

model = spectral_normalize(model)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('training model...')
for epoch in range(240):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 8 == 7:
        test(model, train_loader)
        test(model, val_loader)

/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]


building model...
32 16
training model...
epoch 0


391it [00:23, 16.93it/s]


...

391it [00:23, 16.97it/s]


epoch 95


391it [00:23, 16.82it/s]



Test set: Avg. loss: 0.0055, Accuracy: 38199/50000 (76%)


Test set: Avg. loss: 0.0108, Accuracy: 5971/10000 (60%)

epoch 96


154it [00:09, 17.38it/s]

In [ ]:
from shearletNN.complex_deit import Attention, vit_models, Block, FreqEmbed, complex_freakformer_small_patch2_LS
from shearletNN.layers import CReLU, ComplexLayerNorm
from functools import partial

def repeat3(x):
    return x.repeat(3, 1, 1)[:3]

train_transform = v2.Compose([
    transforms.RandomResizedCrop((image_size, image_size), scale=(0.5, 1.0)),
    # transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = v2.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

ds_train = torchvision.datasets.CIFAR10('../', transform=train_transform, download = True, train=True)
ds_train = IndexSubsetDataset(ds_train, sum([list(range(len(ds_train)))[i::5] for i in range(1, 5)], []))

ds_val = torchvision.datasets.CIFAR10('../', transform=val_transform, download = True, train=False)
ds_val = IndexSubsetDataset(ds_val, list(range(len(ds_val)))[0::5])

train_loader = torch.utils.data.DataLoader(
  ds_train,
  batch_size=batch_size_train, shuffle=True, num_workers=0)


val_loader = torch.utils.data.DataLoader(
  ds_val,
  batch_size=batch_size_train, shuffle=False)


model = torchvision.models.resnet18()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('training model...')
for epoch in range(128):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()

    test(model, train_loader)
    test(model, val_loader)

In [ ]:
print('training model...')
for epoch in range(256):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 16 == 15:
        test(model, train_loader)
        test(model, val_loader)

In [6]:
from shearletNN.complex_deit import vit_models, LPatchEmbed, Attention, Block, ComplexLayerNorm
from shearletNN.layers import CReLU, ComplexLayerNorm
from functools import partial

patch_size = 64
image_size = 128

def repeat3(x):
    return x.repeat(3, 1, 1)[:3]

train_transform = v2.Compose([
    transforms.RandomResizedCrop((image_size, image_size), scale=(0.5, 1.0)),
    # transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = v2.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

ds_train = torchvision.datasets.Caltech101('./', transform=train_transform, download = True)
ds_train = IndexSubsetDataset(ds_train, sum([list(range(len(ds_train)))[i::5] for i in range(1, 5)], []))

ds_val = torchvision.datasets.Caltech101('./', transform=val_transform, download = True)
ds_val = IndexSubsetDataset(ds_val, list(range(len(ds_val)))[0::5])

train_loader = torch.utils.data.DataLoader(
  ds_train,
  batch_size=batch_size_train, shuffle=True, num_workers=0)

shearlets = shearlets[:3]

def shearlet_transform(img):
    return shifted_frequency_shearlet_transform(img.to(0), shearlets.to(0), image_size)

train_loader = ShearletTransformLoader(train_loader, shearlet_transform)

val_loader = torch.utils.data.DataLoader(
  ds_val,
  batch_size=batch_size_train, shuffle=False)

val_loader = ShearletTransformLoader(val_loader, shearlet_transform)

for x, y in tqdm(train_loader):
    assert list(x.shape) == [batch_size_train, shearlets.shape[0] * 3, image_size, image_size], x.shape
    break
print('building model...')

model = vit_models(
        in_chans=shearlets.shape[0] * 3,
        img_size=image_size,
        patch_size=32,
        embed_dim=384,
        depth=12,
        num_heads=6,
        mlp_ratio=4,
        qkv_bias=True,
        norm_layer=partial(ComplexLayerNorm, eps=1e-6),
        block_layers=Block,
        Attention_block=Attention,
        act_layer=CGELU,
        Patch_layer=LPatchEmbed,
    )


with torch.no_grad():
    for param in model.parameters():
        if not ((param == 0).all() or (param == 1).all()):
            param.data /= 2**(0.5)

model = spectral_normalize(model)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('training model...')
for epoch in range(128):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=2)
    gc.collect()
    if epoch % 8 == 7:
        test(model, train_loader)
        test(model, val_loader)

/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]


building model...
training model...
epoch 0


55it [00:09,  5.82it/s]


...

epoch 126


55it [00:09,  5.96it/s]


epoch 127


55it [00:09,  5.89it/s]



Test set: Avg. loss: 1.5677, Accuracy: 1779/6941 (26%)


Test set: Avg. loss: 2.1764, Accuracy: 290/1736 (17%)



In [9]:
for epoch in range(256):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=2)
    gc.collect()
    if epoch % 8 == 7:
        test(model, train_loader)
        test(model, val_loader)

epoch 0


55it [00:09,  5.90it/s]


epoch 1


55it [00:09,  6.06it/s]


epoch 2


...


Test set: Avg. loss: 0.0216, Accuracy: 2896/6941 (42%)


Test set: Avg. loss: 0.0302, Accuracy: 496/1736 (29%)

epoch 104


55it [00:08,  6.15it/s]


epoch 105


47it [00:07,  5.95it/s]


KeyboardInterrupt: 

/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/jroth/miniforge3/envs/shearlets/lib/python3.9/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]


building model...
training model...
epoch 0


0it [00:00, ?it/s]


RuntimeError: The size of tensor a (4) must match the size of tensor b (16) at non-singleton dimension 1

In [ ]:
# amplitude:
amp = torch.abs(x)
print((amp.real.max(), amp.real.min()))

phase = torch.arctan(x.imag / x.real)

phase = torch.nan_to_num(torch.arctan(x.imag / x.real), posinf=torch.math.pi / 2, neginf= -torch.math.pi / 2)

print((phase.real.max(), phase.real.min()))

def to_magnitude_phase(x):
    """
    return magnitude/phase representation 
    """
    return torch.complex(torch.abs(x), torch.nan_to_num(torch.arctan(x.imag / x.real), posinf=torch.math.pi / 2, neginf= -torch.math.pi / 2))

(tensor(5.4802, device='cuda:0'), tensor(0., device='cuda:0'))
(tensor(1.5708, device='cuda:0'), tensor(-1.5708, device='cuda:0'))


In [ ]:
x = torch.tensor([[1.5,.0,.0,.0]])
layerNorm = torch.nn.LayerNorm(4, elementwise_affine = False)
y1 = layerNorm(x)
mean = x.mean(-1, keepdim = True)
var = x.var(-1, keepdim = True, unbiased=False)
y2 = (x-mean)/torch.sqrt(var+layerNorm.eps)

torch.allclose(y1, y2)

True

In [ ]:
x = torch.randn((50,20,100))

layerNorm = torch.nn.LayerNorm(x.shape[-1], elementwise_affine = True)
y1 = layerNorm(x)

mean = torch.mean(x, dim=-1, keepdim=True)
var = torch.square(x - mean).mean(dim=-1, keepdim=True)
y2 = ((x - mean) / torch.sqrt(var + layerNorm.eps)) * layerNorm.weight + layerNorm.bias

torch.allclose(y1, y2, atol=1e-5, rtol=1e-5)

True